# Notebook 4: Tax Strategies - RMDs and Step-Up in Basis

**Learning Objectives:**
- Calculate Required Minimum Distributions (RMDs)
- Understand the step-up in basis at death
- Compare Hold-to-Death vs Aggressive Conversion strategies
- See how tax rules create optimization opportunities

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

## Required Minimum Distributions (RMDs)

Starting at age 73, the IRS requires you to withdraw a minimum amount from Traditional IRAs each year.

**RMD = Account Balance / Distribution Period**

The distribution period comes from the IRS Uniform Lifetime Table.

In [ ]:
# IRS Uniform Lifetime Table (2024)
RMD_DIVISORS = {
    72: 27.4, 73: 26.5, 74: 25.5, 75: 24.6, 76: 23.7, 77: 22.9, 78: 22.0,
    79: 21.1, 80: 20.2, 81: 19.4, 82: 18.5, 83: 17.7, 84: 16.8, 85: 16.0,
    86: 15.2, 87: 14.4, 88: 13.7, 89: 12.9, 90: 12.2, 91: 11.5, 92: 10.8,
    93: 10.1, 94: 9.5, 95: 8.9, 96: 8.4, 97: 7.8, 98: 7.3, 99: 6.8, 100: 6.4
}

def calculate_rmd(ira_balance, age):
    """Calculate Required Minimum Distribution."""
    if age < 73:
        return 0.0
    divisor = RMD_DIVISORS.get(age, max(2.0, 6.4 - (age - 100) * 0.4))
    return ira_balance / divisor

# Example: RMD on $1M IRA at different ages
print("RMD on $1,000,000 IRA by Age")
print(f"{'Age':>4} {'Divisor':>8} {'RMD':>12} {'% of Balance':>14}")
for age in [73, 75, 80, 85, 90, 95]:
    rmd = calculate_rmd(1_000_000, age)
    div = RMD_DIVISORS.get(age, 6.4)
    print(f"{age:>4} {div:>8.1f} ${rmd:>11,.0f} {rmd/1e6*100:>13.1f}%")

## The Step-Up in Basis

When you die, your heirs inherit taxable assets at their current market value ("stepped-up basis").

**Example:**
- You bought stock for $100,000 (your basis)
- It's now worth $500,000
- If you sell: pay capital gains tax on $400,000 gain
- If you die: heirs inherit at $500,000 basis, pay $0 tax

This creates an incentive to hold appreciated assets until death.

In [ ]:
def calculate_step_up_benefit(market_value, cost_basis, cap_gains_rate=0.15):
    """Calculate tax savings from step-up in basis."""
    unrealized_gain = max(0, market_value - cost_basis)
    taxes_avoided = unrealized_gain * cap_gains_rate
    return taxes_avoided

# Example
market_value = 500_000
cost_basis = 100_000
benefit = calculate_step_up_benefit(market_value, cost_basis)
print(f"Market Value: ${market_value:,}")
print(f"Cost Basis:   ${cost_basis:,}")
print(f"Unrealized Gain: ${market_value - cost_basis:,}")
print(f"Step-Up Benefit: ${benefit:,} (taxes avoided)")

## Strategy 1: Hold to Death

- Take only required RMDs from IRA
- Never sell taxable assets
- Maximize step-up benefit for heirs

In [ ]:
def simulate_hold_to_death(initial_ira, initial_taxable, basis, start_age, death_age,
                           market_returns, tax_bracket=0.24, cap_gains_rate=0.15):
    """Simulate Hold-to-Death strategy."""
    ira, taxable = initial_ira, initial_taxable
    total_taxes, total_rmds = 0.0, 0.0
    
    for year in range(death_age - start_age):
        age = start_age + year + 1
        ret = market_returns[year] if year < len(market_returns) else 0.0
        ira *= (1 + ret)
        taxable *= (1 + ret)
        
        rmd = calculate_rmd(ira, age)
        if rmd > 0:
            ira -= rmd
            total_taxes += rmd * tax_bracket
            total_rmds += rmd
    
    # Terminal wealth
    ira_after_tax = ira * (1 - tax_bracket)
    step_up = calculate_step_up_benefit(taxable, basis, cap_gains_rate)
    terminal = ira_after_tax + taxable
    
    return terminal, total_taxes, total_rmds, step_up

## Strategy 2: Aggressive Roth Conversion

- Convert $X per year from Traditional IRA to Roth IRA before RMDs start
- Pay taxes now at known rates
- Roth grows tax-free and has no RMDs

In [ ]:
def simulate_aggressive_conversion(initial_ira, initial_taxable, basis, start_age, death_age,
                                   market_returns, tax_bracket=0.24, cap_gains_rate=0.15,
                                   annual_conversion=100_000, conversion_end_age=72):
    """Simulate Aggressive Roth Conversion strategy."""
    ira, roth, taxable = initial_ira, 0.0, initial_taxable
    total_taxes, total_rmds = 0.0, 0.0
    
    for year in range(death_age - start_age):
        age = start_age + year + 1
        ret = market_returns[year] if year < len(market_returns) else 0.0
        ira *= (1 + ret)
        roth *= (1 + ret)
        taxable *= (1 + ret)
        
        # Roth conversion before RMD age
        if age <= conversion_end_age and ira > 0:
            convert = min(annual_conversion, ira)
            ira -= convert
            roth += convert
            total_taxes += convert * tax_bracket
        
        # RMD on remaining IRA
        rmd = calculate_rmd(ira, age)
        if rmd > 0:
            ira -= rmd
            total_taxes += rmd * tax_bracket
            total_rmds += rmd
    
    # Terminal wealth
    ira_after_tax = ira * (1 - tax_bracket)
    step_up = calculate_step_up_benefit(taxable, basis, cap_gains_rate)
    terminal = ira_after_tax + roth + taxable  # Roth is tax-free
    
    return terminal, total_taxes, total_rmds, step_up

In [ ]:
# Compare strategies with fixed returns
returns = np.full(35, 0.07)  # 7% every year

hold = simulate_hold_to_death(1_000_000, 500_000, 200_000, 65, 85, returns)
conv = simulate_aggressive_conversion(1_000_000, 500_000, 200_000, 65, 85, returns)

print("Strategy Comparison (Death at 85, 7% annual returns)")
print(f"{'':>25} {'Hold-to-Death':>15} {'Aggressive Conv':>15}")
print(f"{'Terminal Wealth':>25} ${hold[0]:>14,.0f} ${conv[0]:>14,.0f}")
print(f"{'Total Taxes Paid':>25} ${hold[1]:>14,.0f} ${conv[1]:>14,.0f}")
print(f"{'Total RMDs':>25} ${hold[2]:>14,.0f} ${conv[2]:>14,.0f}")
print(f"{'Step-Up Benefit':>25} ${hold[3]:>14,.0f} ${conv[3]:>14,.0f}")

## Summary

- RMDs force withdrawals from Traditional IRAs starting at 73
- Step-up in basis eliminates capital gains tax at death
- Different strategies optimize for different scenarios
- The "best" strategy depends on lifespan and market returns

**Next: Notebook 5 - Full Monte Carlo Simulation**